In [ ]:
# Boilerplate: This block goes into every notebook.
# It sets up the environment, installs the requirements, and checks for the required environment variables.

from IPython.display import clear_output
from dotenv import load_dotenv
import os

requirements_installed = False
max_retries = 3
retries = 0
REQUIRED_ENV_VARS = ["OPENAI_API_KEY"]


def install_requirements():
    """Installs the requirements from requirements.txt file"""
    global requirements_installed
    if requirements_installed:
        print("Requirements already installed.")
        return

    print("Installing requirements...")
    install_status = os.system("pip install -r requirements.txt")
    if install_status == 0:
        print("Requirements installed successfully.")
        requirements_installed = True
    else:
        print("Failed to install requirements.")
        if retries < max_retries:
            print("Retrying...")
            retries += 1
            return install_requirements()
        exit(1)
    return


def setup_env():
    """Sets up the environment variables"""

    def check_env(env_var):
        value = os.getenv(env_var)
        if value is None:
            print(f"Please set the {env_var} environment variable.")
            exit(1)
        else:
            print(f"{env_var} is set.")

    load_dotenv(override=True)

    variables_to_check = REQUIRED_ENV_VARS

    for var in variables_to_check:
        check_env(var)


install_requirements()
clear_output()
setup_env()
print("🚀 Setup complete. Continue to the next cell.")

In [ ]:
## Simple PubMed Tool with LangChain Agent

from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.pubmed.tool import PubmedQueryRun
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o")

pubmed_tool = PubmedQueryRun()

tools = [pubmed_tool]

messages = [
    (
        "ai",
        "You are an AI called SV that's is designed to find cures for diseases. You have access to a database of scientific papers and can ask for more information if needed.",
    ),
    (
        "user",
        "{question}. Provide the names of references in the bracket whenever you mention a fact.",
    ),
    ("placeholder", "{agent_scratchpad}"),
]

question = "How can COVID-19 be treated?"

prompt = ChatPromptTemplate.from_messages(messages)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

result = agent_executor.invoke({"question": question})

print(result.get("output"))